In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 15)

LARGE_FIGSIZE = (12, 8)

matplotlib = 3.3.2

numpy = 1.19.2

pandas = 1.1.3

psycopg2 = 2.8.5 

python = 3.8.5

In [3]:
df = pd.read_csv('C:/Users/Kate/PycharmProjects/test/test/P9-ConsumerComplaints.csv',
                 header = 0,
                 na_values=['N/A'],
                 index_col = 'Complaint ID',
                 true_values=['Yes'],
                 false_values=['No'],
                 #keep_default_na=False,
                 low_memory=False
                 )
df.head()

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed
Complaint ID,,,,,,,,,,,,,,,,,
468882,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,True,False
468889,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,True,False
468879,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,True,False
468949,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False
475823,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,True,False


In [2]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context
# df = pd.read_csv('https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P9-ConsumerComplaints.csv')


C:\Users\Kate\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df['Date Received'] = pd.to_datetime(df['Date Received'])

In [5]:
df.dtypes

Date Received                   datetime64[ns]
Product Name                            object
Sub Product                             object
Issue                                   object
Sub Issue                               object
                                     ...      
Submitted via                           object
Date Sent to Company                    object
Company Response to Consumer            object
Timely Response                           bool
Consumer Disputed                       object
Length: 17, dtype: object

In [6]:
date_start = pd.to_datetime('2013-07-29')
date_end = pd.to_datetime('2013-07-30')

In [7]:
# all product names
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)]['Product Name'].unique()

array(['Consumer Loan', 'Bank account or service', 'Mortgage',
       'Debt collection', 'Credit card', 'Credit reporting',
       'Student loan'], dtype=object)

In [8]:
# complaints for each product
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end)][['Product Name','Issue']].sort_values('Product Name')

,Product Name,Issue
Complaint ID,,
469537,Bank account or service,Problems caused by my funds being low
470611,Bank account or service,"Account opening, closing, or management"
479508,Bank account or service,Problems caused by my funds being low
469446,Bank account or service,Deposits and withdrawals
470633,Bank account or service,"Account opening, closing, or management"
...,...,...
469340,Student loan,Repaying your loan
467427,Student loan,Repaying your loan
479648,Student loan,Repaying your loan


In [9]:
# how much with timely response
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end) & (df['Timely Response'] == True)][['Product Name','Timely Response']].groupby('Product Name').sum()

,Timely Response
Product Name,
Bank account or service,34
Consumer Loan,10
Credit card,26
Credit reporting,22
Debt collection,25
Mortgage,75
Student loan,7


In [10]:
# how much disputed by customer
df[(df['Date Received'] >= date_start) & (df['Date Received'] <= date_end) & (df['Consumer Disputed'] == True)][['Product Name','Consumer Disputed']].groupby('Product Name').sum()

,Consumer Disputed
Product Name,
Bank account or service,3
Consumer Loan,3
Credit card,10
Credit reporting,2
Debt collection,3
Mortgage,16


In [11]:
df=df.reset_index()

In [12]:
df[df['Timely Response']==True ].groupby(['Product Name','Issue']).sum().sort_values('Product Name', ascending=True)


Complaint ID  \
Product Name            Issue                                                    
Bank account or service Account opening, closing, or management     2604821851   
                        Deposits and withdrawals                    1567539470   
                        Making/receiving payments, sending money     530539712   
                        Problems caused by my funds being low        759833797   
                        Using a debit or ATM card                    432069232   
...                                                                        ...   
Student loan            Problems when you are unable to pay           41388878   
                        Can't repay my loan                          618635474   
                        Dealing with my lender or servicer           853631569   
                        Getting a loan                                50095982   
                        Repaying your loan                           127175019   

                                                                  Timely Response  
Product Name            Issue                                                      
Bank account or service Account opening, closing, or management              2883  
                        Deposits and withdrawals                             1745  
                        Making/receiving payments, sending money              568  
                        Problems caused by my funds being low                 837  
                        Using a debit or ATM card                             461  
...                                                                           ...  
Student loan            Problems when you are unable to pay                    75  
                        Can't repay my loan                                   629  
                        Dealing with my lender or servicer                    820  
                        Getting a loan                                         53  
                        Repaying your loan                                    229  

[100 rows x 2 columns]

In [13]:
df[df['Consumer Disputed']==True ].groupby(['Product Name','Issue']).sum().sort_values('Product Name', ascending=True)

Complaint ID  \
Product Name            Issue                                                    
Bank account or service Account opening, closing, or management      533819018   
                        Deposits and withdrawals                     291355594   
                        Making/receiving payments, sending money     101534017   
                        Problems caused by my funds being low        113825733   
                        Using a debit or ATM card                     76136236   
...                                                                        ...   
Student loan            Problems when you are unable to pay            8324242   
                        Can't repay my loan                           80323504   
                        Dealing with my lender or servicer           203361766   
                        Getting a loan                                 9792292   
                        Repaying your loan                            26080020   

                                                                  Timely Response  
Product Name            Issue                                                      
Bank account or service Account opening, closing, or management               590  
                        Deposits and withdrawals                              328  
                        Making/receiving payments, sending money              104  
                        Problems caused by my funds being low                 118  
                        Using a debit or ATM card                              82  
...                                                                           ...  
Student loan            Problems when you are unable to pay                    15  
                        Can't repay my loan                                    80  
                        Dealing with my lender or servicer                    190  
                        Getting a loan                                         10  
                        Repaying your loan                                     47  

[93 rows x 2 columns]

In [80]:
import psycopg2
connection = psycopg2.connect(user="postgres",
                              password="o0pO)P",
                              host="127.0.0.1",
                              port="5433",
                              database="postgres")

cursor = connection.cursor()

In [31]:
def issuecounter(cursor, date_from, date_to):
    query = """
        select  product_name, issue,
        count(timely_response) filter (where timely_response_bool is true) trc,
        count(consumer_disputed) filter (where consumer_disputed_bool is true) cdc
        from user_complaints
        where date_received >= %s and date_received <= %s
        group by product_name, issue
        order by product_name, issue, count(issue) desc
        
    ;"""
    cursor.execute(query, (date_from, date_to))
    header = "product_name, issues, count_timely_response, count_consumer_disputed"
    print(header)
    for row in cursor:
        print(row)

In [32]:
%%timeit
issuecounter(cursor, '2013-07-29', '2013-07-30')

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

...

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

product_name, issues, count_timely_response, count_consumer_disputed
('Bank account or service', 'Account opening, closing, or management', 12, 3)
('Bank account or service', 'Deposits and withdrawals', 13, 0)
('Bank account or service', 'Making/receiving payments, sending money', 2, 0)
('Bank account or service', 'Problems caused by my funds being low', 4, 0)
('Bank account or service', 'Using a debit or ATM card', 3, 0)
('Consumer Loan', 'Managing the loan or lease', 7, 3)
('Consumer Loan', 'Problems when you are unable to pay', 2, 0)
('Consumer Loan', 'Shopping for a loan or lease', 1, 0)
('Credit card', 'Advertising and marketing', 2, 0)
('Credit card', 'Application processing delay', 1, 0)
('Credit card', 'APR or interest rate', 4, 1)
('Credit card', 'Billing disputes', 6, 2)
('Credit card', 'Billing statement', 1, 1)
('Credit card', 'Closing/Cancelling account', 2, 1)
('Credit card', 'Credit determination', 2, 1)
('Credit card', 'Credit line increase/decrease', 1, 1)
('Credit car

In [81]:
if (connection):
    cursor.close()
    connection.close()

NameError: name 'date_start' is not defined